In [ ]:
import copy
import os
import yaml
import sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob


font = {
    "family": 'monospace',
    "size": 20
}
matplotlib.rc('font', **font)

# Main data path
data_path = os.path.join(os.getcwd(), "data")
# Figures save paths
with open("secrets.yml", "r") as file_handle:
    fig_save_paths = yaml.load(file_handle, Loader=yaml.Loader)

# Assessment of Lifting

- The task involves lifting set weights
    - The lifting sequence - DAQ start, wait 5s, lift weight, hold, wait 5s 
- Analyzing the data from lifting weights
- Multiple individuals were used in the data collection

## Loading data

In [ ]:
item_considered = "CardboardBox"

In [ ]:
weight_lifting = os.path.join(data_path, "LiftingAssessment")
# Get all the ".csv" files
all_parsed_files = glob.glob("**/*.csv", root_dir=weight_lifting, recursive=True)

# Select Task-2
selected_parsed_files = [x for x in all_parsed_files if item_considered in x.split(os.sep)]

# Get the weight classes
weights = []
for file in selected_parsed_files:
    weight_class = file.split(os.sep)[2]
    if weight_class not in weights:
        weights.append(weight_class)

# Load the data
loaded_data = {}
for file_path in selected_parsed_files:
    # Full path to file
    full_path = os.path.join(weight_lifting, file_path)

    # Load the time of DAQ
    with open(full_path, "r") as file_handle:
        daq_time = file_handle.readline()
        daq_time = daq_time.split(" ")[-1]
        daq_time = int(daq_time[0:-2])
    # Read the csv
    df = pd.read_csv(full_path, header="infer", skiprows=1)

    # Store data
    loaded_data[full_path] = {
        "daq_time": daq_time,
        "df": df
    }


### Evaluate the Sampling Performance

How effectively can we maintain the sampling rate?


In [ ]:
sentinels = {"DAQSentinel01": [],
             "DAQSentinel02": [],
             "DAQSentinel03": []}
sampling_rates = {}
for file_path, data in loaded_data.items():
    # Choose the right sentinel
    sentinel = file_path.split("/")[-1].split("_")[0]

    # Determine sampling rate
    total_time = data["daq_time"]
    samples = data["df"].shape[0]
    sentinels[sentinel].append(samples / total_time)

for sentinel in sentinels.keys():
    print("Sampling Rate for " + sentinel + " with mean " + str(round(np.mean(sentinels[sentinel]), 2)) +
          " and std of " + str(round(np.std(sentinels[sentinel]), 2)))

    # Get the mean sampling rate
    sampling_rates[sentinel] = round(np.mean(sentinels[sentinel]), 2)

### Plot01 - Raw Time Series Plotting

Plotting the raw time series data for a selected trial and a selected sensor-data type

In [ ]:
# Plot Configuration
sensor_type = "acc"
individual = "Aditya"
sentinel = "DAQSentinel01"
sampling_rate = sampling_rates[sentinel]
trial = "Trial-5"

In [ ]:
# Create figure and axes
fig = plt.figure(figsize=(25, 20))
axs = [fig.add_subplot(5, 1, x+1) for x in range(5)]

# Select specific items
selected_loaded_data = {}
for file_name in loaded_data.keys():
    if ((sentinel + "_dataAGO.csv") in file_name.split(os.sep)) and (trial in file_name.split(os.sep)) and (individual in file_name.split(os.sep)):
        selected_loaded_data[file_name] = copy.deepcopy(loaded_data[file_name])

# Plot the raw data
for index, data in enumerate(sorted(list(selected_loaded_data.keys()), key=lambda l: int(l.split(os.sep)[-3][1:]))):
    df = selected_loaded_data[data]["df"]
    df["sampleTime"] = list(range(0, df.shape[0]))
    df["sampleTime"] /= sampling_rate
    # Select columns
    cols = [col for col in df.columns.tolist() if sensor_type in col]
    cols.append("sampleTime")

    # Plot raw time series
    df[cols].plot(ax=axs[index], x="sampleTime")

    # Label the plot
    axs[index].set_title(data.split(os.sep)[-3])
    axs[index].set_xlabel("Time(s)")
    if sensor_type == "acc":
        axs[index].set_ylabel("Acceleration(g)")
        axs[index].set_ylim([-1.5, 0.5])
        axs[index].set_xlim([0, 15])
    elif sensor_type == "gyr":
        axs[index].set_ylabel("Gyroscope(deg/s)")
        axs[index].set_ylim([-40, +40])
        axs[index].set_xlim([0, 15])

fig.tight_layout()
fig.savefig(os.path.join(fig_save_paths["save_paths"]["plot1"], sentinel + "-" + individual + "-" + item_considered + "-" + sensor_type + ".png"), dpi=600)
plt.show()

#### Insights

1. As the weight increases, we can see high frequency components in the acceleration sensor signal for the sentinels located at the wrist
    1. The pattern can be seen across both crate and the cardboard box
2. For the case of Sentinel02, which is located in the lumbosacral region (lumbar spine), the high frequency components can be seen as the weight increases, but it is not as prominent as the other two sentinels on the wrist
3. For cardboard box lift, there is a big swing in the Z-axis acceleration. The swing is not so prominent on the Crate lift. Maybe this is due to the size of the box?


### Plot02 - Histogram of the sensor output

Where do the sensor values lie in general, across all the trials and by class?

Structure of the plots
- Create plots for all Sentinels across all five trials, and seperated by class within a plot.

In [ ]:
individual = "Aditya"
item_considered = "Crate"
seconds_cropped = 4

In [ ]:
sensor_types = ["acc", "gyr"]
for sensor_type in sensor_types:
    for sentinel in sentinels.keys():
        # Get all data for a sentinel
        df_keys_by_sentinel = [x for x in loaded_data.keys() if sentinel in x]
        
        # Get the weight classes
        classes = list(set(sorted([x.split(os.sep)[-3] for x in df_keys_by_sentinel])))
        classes = sorted(classes, key=lambda l: int(l[1:]))
        # Create a certain number of plots
        fig = plt.figure(figsize=(25, 20))
        axs = [fig.add_subplot(len(classes), 1, x+1) for x in range(len(classes))]
        
        # Group data by class
        for class_index, class_instance in enumerate(classes):
            # Select all trials for a class instance
            df_keys_by_sentinelClass = [x for x in df_keys_by_sentinel if class_instance in x and individual in x and item_considered in x]
            # Group df by class
            for index, item in enumerate(df_keys_by_sentinelClass):
                # Get the data
                if index == 0:
                    df = loaded_data[item]["df"]
                    df = df.iloc[int(seconds_cropped * sampling_rates[sentinel]):int(df.shape[0] - (seconds_cropped * sampling_rates[sentinel]))]
                else:
                    temp = loaded_data[item]["df"]
                    temp = temp.iloc[int(seconds_cropped * sampling_rates[sentinel]):int(temp.shape[0] - (seconds_cropped * sampling_rates[sentinel]))]
                    df = pd.concat([df, temp], ignore_index=True, copy=True)
                
            # Select only the required columns
            cols = [col for col in df.columns.tolist() if sensor_type in col]
            # Creating the histogram plot
            for col in cols:
                axs[class_index].hist(df[col], bins=100, label=col)
                # Label the figures
                axs[class_index].set_title(class_instance)
                if sensor_type == "acc":
                    axs[class_index].set_xlabel("Acceleration(g)")
                elif sensor_type == "gyr":
                    axs[class_index].set_xlabel("Gyroscope(deg/s)")
                axs[class_index].set_ylabel("Frequency")
                axs[class_index].legend()
            # Share the x-axis
            for i in range(1, len(axs)):
                axs[i-1].sharex(axs[i])
            axs[class_index].set_ylim([0, 6000])
                
        # Save the figure
        fig.tight_layout()
        fig.savefig(os.path.join(fig_save_paths["save_paths"]["plot2"], sentinel + "-" + individual + "-" + item_considered + "-" + sensor_type + ".png"), dpi=600)
        
        # Clear a few items
        del fig, df
    